# QUBO optimization on a qumode

## Prerequisite

Cell for Google Colab users.

In [ ]:
!pip install git+https://github.com/sympy/sympy.git
!pip install qutip
!pip install scipy

  Cloning https://github.com/sympy/sympy.git to /tmp/pip-req-build-n3ddi47n
  Running command git clone --filter=blob:none --quiet https://github.com/sympy/sympy.git /tmp/pip-req-build-n3ddi47n
  Resolved https://github.com/sympy/sympy.git to commit 91ab0d56601537db30574666941424d4c24f3083
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sympy: filename=sympy-1.14.dev0-py3-none-any.whl size=6248970 sha256=ed44ec313a53ae774253b401bf737e32aaef41e8d5418bbbcd30b2d1e4e0e9a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-fdmkrjda/wheels/77/06/c3/4de376dd4507851c07f5b7581d16e85d493a4ec0f0adbc6423
Successfully built sympy
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [ ]:
import numpy as np
import qutip as qt
import sympy as sp
import scipy.optimize as sciopt

from functools import partial

In [ ]:
import matplotlib.pyplot as plt

## Ansatz

### Basics

In [ ]:
def get_cvec_np(r, theta):
    r = np.array(r)
    theta = np.array(theta)
    return r * np.exp(1j * theta)

In [ ]:
def pack_variables(beta_mag, beta_arg, theta, phi):
    Xvec = np.concatenate([
        beta_mag.ravel(),
        beta_arg.ravel(),
        theta.ravel(),
        phi.ravel()
    ])
    return Xvec


def unpack_variables(Xvec, ndepth):
    size = ndepth * 2

    beta_mag = Xvec[:size].reshape((ndepth, 2))
    beta_arg = Xvec[size:2*size].reshape((ndepth, 2))
    theta = Xvec[2*size:3*size].reshape((ndepth, 2))
    phi = Xvec[3*size:].reshape((ndepth, 2))

    return beta_mag, beta_arg, theta, phi

In [ ]:
def qproj00():
    return qt.basis(2, 0).proj()


def qproj11():
    return qt.basis(2, 1).proj()


def qproj01():
    op = np.array([[0, 1], [0, 0]])
    return qt.Qobj(op)


def qproj10():
    op = np.array([[0, 0], [1, 0]])
    return qt.Qobj(op)

### ECDs with qubit rotations

Qubit rotation with qumode echoed conditional displacement (ECD) operators ([reference](https://doi.org/10.1038/s41567-022-01776-9)) for one qubit and two qumodes

\begin{align*}
U (\vec{\beta}, \vec{\theta}, \vec{\phi})
&= ECD_2 (\beta_2) \:
\big[ R (\theta_2, \phi_2) \otimes I \otimes I \big] \:
ECD_1 (\beta_1) \:
\big[ R (\theta_1, \phi_1) \otimes I \otimes I \big]
\\
ECD_1 (\beta_1)
&= |1 \rangle \langle 0| \otimes D (\beta_1 / 2) \otimes I  
+ |0 \rangle \langle 1| \otimes D (-\beta_1 / 2) \otimes I,
\\
ECD_2 (\beta_2)
&= |1 \rangle \langle 0| \otimes I \otimes D (\beta_2 / 2)
+ |0 \rangle \langle 1| \otimes I \otimes D (-\beta_2 / 2),
\end{align*}

where
$ R (\theta, \phi)
= e^{ - i (\theta / 2) \big[ \cos(\phi) X + \sin(\phi) Y \big] } $ and
$ D (\beta) = e^{ \beta a^\dagger - \beta^* a } $.

In [ ]:
def qubit_rot(theta, phi):
    """
    R (theta, phi) = exp[ −i (theta/2) ( X cos(phi) + Y sin(phi) ) ].

    Arguments:
    theta, phi: rotation parameters
    """
    gen = ( qt.sigmax() * np.cos(phi) )
    gen += ( qt.sigmay() * np.sin(phi) )

    H = -1j * (theta / 2) * gen

    return H.expm()

In [ ]:
def ecd_op(beta, theta, phi, cind, nfocks):
    """
    ECD operator.

    Arguments:
    beta -- ECD parameter
    theta, phi -- rotation parameters
    cind -- qumode index
    nfocks -- Fock cutoffs
    """
    # Validate cind
    if cind not in (0, 1):
        raise ValueError("cind must be 0 or 1")

    # ECD
    if cind == 0:
        E2 = qt.tensor(qproj10(), qt.displace(nfocks[0], beta / 2))
        E2 += qt.tensor(qproj01(), qt.displace(nfocks[0], -beta / 2))
        E2 = qt.tensor(E2, qt.qeye(nfocks[1]))
    else:
        E2 = qt.tensor(qproj10(), qt.qeye(nfocks[0]), qt.displace(nfocks[1], beta / 2))
        E2 += qt.tensor(qproj01(), qt.qeye(nfocks[0]), qt.displace(nfocks[1], -beta / 2))

    return E2

In [ ]:
def ecd_rot_op(beta, theta, phi, nfocks):
    """
    ECD-rotation operator.

    Arguments:
    beta -- ECD parameters
    theta, phi -- rotation parameters
    nfocks -- Fock cutoffs
    """
    # Rotations
    R1 = qt.tensor(qubit_rot(theta[0], phi[0]), qt.qeye(nfocks[0]), qt.qeye(nfocks[1]))
    R2 = qt.tensor(qubit_rot(theta[1], phi[1]), qt.qeye(nfocks[0]), qt.qeye(nfocks[1]))

    # ECDs
    E1 = ecd_op(beta[0], theta[0], phi[0], 0, nfocks)
    E2 = ecd_op(beta[1], theta[1], phi[1], 1, nfocks)

    return E2 * R2 * E1 * R1

Build the ansatz matrix of depth $N_d$

$$ \mathcal{U} (\bar{\beta}, \bar{\theta}, \bar{\phi})
= U (\vec{\beta}_{N_d}, \vec{\theta}_{N_d}, \vec{\phi}_{N_d}) \cdots
U (\vec{\beta}_1, \vec{\theta}_1, \vec{\phi}_1),
$$

where $\bar{\beta}, \bar{\theta}$, and $\bar{\phi}$ are matrices of dimensions $N_d \times 2$.
The matrix $\bar{\beta}$ is also complex-valued.

In [ ]:
def ecd_rot_ansatz(bmag_mat, barg_mat, theta_mat, phi_mat, nfocks):
    """
    ECD-rotation ansatz.

    Arguments:
    bmag_mat, barg_mat -- ECD parameters
    theta_mat, phi_mat -- rotation parameters
    nfocks -- Fock cutoffs
    """
    # Check
    if bmag_mat.shape != barg_mat.shape:
        raise ValueError("Dimensions of bmag_mat and barg_mat do not match.")
    beta_mat = get_cvec_np(bmag_mat, barg_mat)
    if beta_mat.shape != theta_mat.shape:
        raise ValueError("Lengths of beta_mat and theta_mat do not match.")
    if beta_mat.shape != phi_mat.shape:
        raise ValueError("Lengths of beta_mat and phi_mat do not match.")

    # Initialize
    ndepth = beta_mat.shape[0]
    uni = ecd_rot_op(beta_mat[0, :], theta_mat[0, :], phi_mat[0, :], nfocks)

    # Check
    if ndepth == 1:
        return uni

    # Loop through blocks
    for i in range(1, ndepth):
        new_uni = ecd_rot_op(beta_mat[i, :], theta_mat[i, :], phi_mat[i, :], nfocks)
        uni = ( new_uni * uni )

    return uni

## Hamiltonian

### Basics

In [ ]:
def decimal_to_binary(decimal_number, length):
    # Convert the decimal number to binary and strip the '0b' prefix
    binary_representation = bin(decimal_number)[2:]

    # Pad the binary representation with leading zeros if necessary
    padded_binary = binary_representation.zfill(length)

    # If the padded length is less than the specified length, raise an error
    if len(padded_binary) > length:
        raise ValueError("The binary representation is longer than the specified length.")

    return padded_binary


def binary_to_decimal(binary_string, length):
    # Validate that the input is a binary string of the specified length
    if len(binary_string) != length:
        raise ValueError(f"Input must be a binary string of length {length}.")

    if not all(bit in '01' for bit in binary_string):
        raise ValueError("Input must be a binary string.")

    # Convert the binary string to decimal
    decimal_number = int(binary_string, 2)

    return decimal_number

In [ ]:
def find_basis_state(state_vector):
    # Convert input to a Qobj if it isn't already
    if not isinstance(state_vector, qt.Qobj):
        state_vector = qt.Qobj(state_vector)

    # Get the number of qubits based on the length of the state vector
    N = int(np.log2(state_vector.shape[0]))

    # Check if the state vector is normalized
    if not np.isclose(state_vector.norm(), 1):
        raise ValueError("Input state vector must be normalized.")

    # Generate all possible basis states for N qubits
    basis_states = [qt.basis(2**N, i) for i in range(2**N)]

    # Check overlap with all basis states
    for index, b in enumerate(basis_states):
        overlap = state_vector.overlap(b)
        if np.isclose(overlap, 1):
            # Convert index to binary representation
            return format(index, f'0{N}b')  # Format as binary string with leading zeros

    # If no overlap found, return None (or raise an error)
    return None

### Qubit

In [ ]:
def binary_to_qubit_ham(H_bin, symbol_list, include_id=False):
    """
    Map a symbolic binary Hamiltonian to a spin Hamiltonian.

    Arguments:
    H_bin -- The binary Hamiltonian as a SymPy object
    symbol_list -- SymPy symbols defining the Hamiltonian
    include_id -- identity as symbol (True) or value 1 (False)
    """
    # Initialize spin variables (Z0, Z1, ..., Zn)
    z_symbols = sp.symbols('z:{}'.format(len(symbol_list)))

    # Define the identity operator (I_j)
    Ident = sp.symbols(r'\mathbb{I}') if include_id else 1.0

    # Create a mapping dictionary from binary symbols to spin expressions
    bin2spin_dict = {
        symbol: (1/2)*(Ident - z) for symbol, z in zip(symbol_list, z_symbols)
    }

    # Convert the binary Hamiltonian to a spin Hamiltonian
    spin_ham = H_bin.subs(bin2spin_dict).expand()

    # Z^2 = I
    sq_z = [z**2 for z in z_symbols]
    sq_values = [Ident] * len(z_symbols)  # All squared terms map to the identity
    spin_squared_map = dict(zip(sq_z, sq_values))

    # Substitute squared terms
    red_spin_ham = spin_ham.subs(spin_squared_map)

    return red_spin_ham


def check_spinz(input_list, spinz):
    out_val = ['I']*len(spinz)
    for ll in range(len(input_list)):
        out_val[int(input_list[ll].strip('z'))] = 'Z'
    return out_val


def sympy_to_pauli_dict(smpy_exp):
    """
    Convert a sympy spin Hamiltonian expression to a dictionary with
    Pauli words as keys and string coefficients as values.
    """
    # Determine the number of qubits
    spinz = smpy_exp.free_symbols

    # Split at spaces so we have the individual terms/coefficients
    split_expr = str(smpy_exp).split()

    # Firs iteration
    matrix_dict = {}
    split_term = split_expr[0].split('*')
    tmp_coeff = split_term[0]
    tmp_paulis = split_term[1:]
    pauli_word = ''.join(check_spinz(tmp_paulis, spinz))
    matrix_dict[pauli_word] = tmp_coeff

    # Iterate through the remaining terms
    for ii in range(1, len(split_expr), 2):
        tmp_sign = split_expr[ii]
        split_term = split_expr[ii+1].split('*')
        tmp_coeff  = split_term[0]
        tmp_paulis = split_term[1:]
        pauli_word = ''.join(check_spinz(tmp_paulis, spinz))
        matrix_dict[pauli_word] = tmp_sign+tmp_coeff

    return matrix_dict


def binary_to_pauli_list(H_total, symbol_list):
    """
    Maps a binary Hamiltonian to Pauli terms and coefficients.

    Arguments:
    H_total -- The binary Hamiltonian
    symbol_list -- symbols defining the Hamiltonian
    """
    spin_ham = binary_to_qubit_ham(H_total, symbol_list)
    op_dict = sympy_to_pauli_dict(spin_ham)

    return [[key, float(value)] for key, value in op_dict.items()]

### Qudit

In [ ]:
def matrices_to_qudit_list(matrices):
    """
    Maps list of matrices to qudit terms.

    Argument:
    matrices -- list of matrices
    """
    # Initialize the result list
    result = []

    # Get the number of matrices
    num_matrices = len(matrices)

    # Create a list to store diagonal values from each matrix
    diagonal_values = []

    # Collect diagonal values and their labels
    for index, matrix in enumerate(matrices):
        diag = np.diagonal(matrix)  # Get the diagonal elements
        diagonal_values.append((diag, index))  # Store as (diagonal_elements, index)

    # Recursive function to generate combinations of indices
    def generate_combinations(combination, depth):
        if depth == num_matrices:
            # Calculate the product for the current combination
            product = 1
            label = []
            for matrix_index, diag_index in combination:
                product *= matrices[matrix_index][diag_index, diag_index]
                label.append(f'P{diag_index}')
            result.append([ ', '.join(label), product ])
            return

        # Loop through the diagonal elements of the current matrix
        diag, matrix_index = diagonal_values[depth]
        for i in range(len(diag)):
            generate_combinations(combination + [(matrix_index, i)], depth + 1)

    # Start generating combinations
    generate_combinations([], 0)

    return result


def partition_string_list(input_list, partition_vector):
    """
    Partition the string in the input list based on the partition vector.

    Arguments:
    input_list -- A list where the first element is a string to partition,
                  and the last element is a value to retain.
    partition_vector -- A list of integers that specifies the lengths of the partitions.

    Returns:
    A new list with the partitioned string and the retained value.
    """
    # Ensure the partition_vector is valid
    string_part = input_list[0]
    total_length = sum(partition_vector)

    if total_length != len(string_part):
        raise ValueError("The sum of the partition vector must equal the length of the string.")

    # Partition the string based on the partition vector
    partitions = []
    start_index = 0

    for size in partition_vector:
        partitions.append(string_part[start_index:start_index + size])
        start_index += size

    # Return the modified list
    return partitions + [input_list[-1]]


def partitioned_pauli_term_to_qudit_term(pterm):
    mat_list = []
    for i in range(len(pterm) - 1):
        new_mat = generate_tensor_product(pterm[i])
        if i == 0:
            new_mat *= pterm[-1]
        mat_list.append(np.real( new_mat.full() ))

    return matrices_to_qudit_list(mat_list)


def pauli_list_to_qudit_terms(pterms, partition_vector):
    term_list = []
    for term in pterms:
        new_term = partition_string_list(term, partition_vector)
        qudit_term = partitioned_pauli_term_to_qudit_term(new_term)
        term_list.append(qudit_term)

    result_dict = {}
    for lst in term_list:
        for key, value in lst:
            if key in result_dict:
                result_dict[key] += value
            else:
                result_dict[key] = value
    return [[key, value] for key, value in result_dict.items()]

### Qutip

In [ ]:
def generate_tensor_product(string):
    """
    Get QuTip object given a string representing a Pauli word.
    """
    # Define a mapping of characters to corresponding QuTiP operators
    operator_map = {
        'I': qt.qeye(2),  # Identity operator
        'X': qt.sigmax(),  # Pauli-X operator
        'Y': qt.sigmay(),  # Pauli-Y operator
        'Z': qt.sigmaz()   # Pauli-Z operator
    }

    # Create a list to collect the operators
    operators = []

    # Append the corresponding operators based on the input string
    for char in string:
        operators.append(operator_map[char])

    # Compute the tensor product of all operators in the list
    U = qt.tensor(*operators).full()

    return qt.Qobj(U)


def qubit_op_to_ham(pterms):
    """
    Get QuTip object given a set of Pauli words and correspdoing coefficients.
    """
    terms = []
    for p in pterms:
        term = ( p[1] * generate_tensor_product(p[0]) )
        terms.append(term)

    return sum(terms)

## Loss function

We want to minimize the following cost function:

$$ \min_{ \overrightarrow{\beta}, \overrightarrow{\theta}, \overrightarrow{\phi} } E
= \langle \psi (\overrightarrow{\beta}, \overrightarrow{\theta}, \overrightarrow{\phi}) | \:
H \: |\psi (\overrightarrow{\beta}, \overrightarrow{\theta}, \overrightarrow{\phi})\rangle,
$$
where $H$ is the two-qumode Hamiltonian and $|\psi \rangle$ is the two-qumode state traced from the ECD-rotation ansatz $|\Psi \rangle$.

In [ ]:
def state_from_ecd(Xvec, ndepth, nfocks):
    """
    Qumode state |Psi> = U ( |0> |0, 0> ).

    Arguments:
    Xvec -- ECD-rotation parameters
    ndepth -- circuit depth
    nfocks -- Fock cutoffs
    """
    # Parameters
    beta_mag, beta_arg, theta, phi = unpack_variables(Xvec, ndepth)

    # ECD unitary
    U = ecd_rot_ansatz(beta_mag, beta_arg, theta, phi, nfocks)

    # U |0, 0, 0>
    vac = qt.tensor( qt.basis(2, 0), qt.basis(nfocks[0], 0), qt.basis(nfocks[1], 0) )
    psi = U * vac

    return psi

In [ ]:
def energy_val(Xvec, ndepth, nfocks, H):
    """
    Compute <psi | H |psi > where

    |psi (n, m)> <== U ( |0> |0, 0> ).

    Arguments:
    Xvec -- ansatz ECD-rotation parameters
    H -- QuTip two-qumode Hamiltonian
    nfock -- Fock cutoff for qumode
    ndepth -- circuit depth
    """
    # Qubit-qubit-qumode state
    psi = state_from_ecd(Xvec, ndepth, nfocks)
    psi = qt.Qobj( psi.full() )

    # Expectation value
    ham = qt.Qobj( H.full() )
    en = qt.expect(ham, psi)

    return en

## Analysis

In [ ]:
def generate_triples(nfocks):
    # Create ranges for q, n, and m
    q_range = np.arange(2)
    n_range = np.arange(nfocks[0])
    m_range = np.arange(nfocks[1])

    # Create a meshgrid of q, n, and m with valid indexing
    q_grid, n_grid, m_grid = np.meshgrid(q_range, n_range, m_range, indexing='ij')

    # Stack the grids to get (q, n, m) triples
    triples = np.stack((q_grid.ravel(), n_grid.ravel(), m_grid.ravel()), axis=-1)

    return triples

In [ ]:
def num_prob_basis(Xvec, nvec, ndepth, nfocks):
    """
    | <psi | q, n, m> |^2, where

    |psi> <== U |0, 0, 0>.

    Arguments:
    Xvec -- ansatz parameters
    nvec -- Fock basis state indices
    nfocks -- Fock cutoffs
    ndepth -- circuit depth
    """
    # Qubit-qubit-qumode state
    psi = state_from_ecd(Xvec, ndepth, nfocks)

    # |q, n, m >
    state = qt.tensor(qt.basis(2, nvec[0]),
                      qt.basis(nfocks[0], nvec[1]),
                      qt.basis(nfocks[1], nvec[2]) )

    # Expectation value
    P0 = psi.overlap(state)

    return np.abs(P0)**2


def num_prob_all(Xvec, ndepth, nfocks):
    """
    | <psi | q, n, m> |^2 for all (n, m).

    Arguments:
    Xvec -- ansatz parameters
    nfock -- Fock cutoff for single qumode
    nvec -- Fock basis state indices
    """
    # Initialize
    N1 = generate_triples(nfocks)
    ntriples = N1.shape[0]

    # Generate
    P1 = []
    for i in range(ntriples):
        P1.append( num_prob_basis(Xvec, N1[i, :], ndepth, nfocks) )

    return np.array(P1)

## Optimization

In [ ]:
def ecd_opt_vqe(H, ndepth, nfocks, maxiter=100, method='COBYLA', verb=0,
                threshold=1e-08, print_freq=10, Xvec=[]):
    """
    Minimize the cost function using SciPy-based methods.

    Arguments:
    H -- QuTip Hamiltonian
    ndepth -- ansatz circuit depth
    nfocks -- Fock cutoffs
    maxiter -- maximum number of iterations
    method -- optimization method
    threshold -- error tolerance
    Xvec -- optional initial guesses
    print_freq -- frequency of printing and storing intermediate results
    """
    # Bound parameters
    beta_mag_min = 0.0
    beta_mag_max = 10.0
    beta_arg_min = 0.0
    beta_arg_max = 2 * np.pi
    theta_min = 0.0
    theta_max = np.pi
    phi_min = 0.0
    phi_max = 2 * np.pi

    # Define bounds
    size = ndepth * 2
    beta_mag_bounds = [(beta_mag_min, beta_mag_max)] * size
    beta_arg_bounds = [(beta_arg_min, beta_arg_max)] * size
    theta_bounds = [(theta_min, theta_max)] * size
    phi_bounds = [(phi_min, phi_max)] * size
    bounds = beta_mag_bounds + beta_arg_bounds + theta_bounds + phi_bounds

    # Guess
    if len(Xvec) == 0:
        beta_mag = np.random.uniform(0, 3, size=(ndepth, 2))
        beta_arg = np.random.uniform(0, np.pi, size=(ndepth, 2))
        theta = np.random.uniform(0, np.pi, size=(ndepth, 2))
        phi = np.random.uniform(0, np.pi, size=(ndepth, 2))
        Xvec = pack_variables(beta_mag, beta_arg, theta, phi)

    # Loss function
    obj_fun = partial(energy_val, ndepth=ndepth, nfocks=nfocks, H=H)

    # Intermediate values
    iteration_step = 0
    intermediate_results = []

    def callback(xk):
        nonlocal iteration_step
        iteration_step += 1
        loss_value = obj_fun(xk)
        if verb == 1 and (iteration_step % print_freq == 0):
            print("-------------------")
            print(f"iter: {iteration_step}")
            print(f"fval: {loss_value}")

        # Store intermediate results
        if iteration_step % print_freq == 0:
            intermediate_results.append((loss_value, xk.copy()))

    # SciPy options
    options = {'disp': True, 'maxiter': maxiter}

    # Optimize
    result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds,
                             tol=threshold, options=options, callback=callback)

    return result.fun, result.x, intermediate_results

## Explore

Let us discuss the following optimization problem

\begin{align*}
\min_{\mathbf{x}} \quad &
x_0 + 2 x_1 + x_2,
\\
\text{subject to} \quad &
x_0 + x_1 = 1,
\\
&2 x_0 + 2 x_1 + x_2 \leq 3,
\\
&x_0 + x_1 + x_2 \geq 1.
\end{align*}
The cost function optimization becomes

\begin{align*}
\min_{\mathbf{x}} F
&= x_0 + x_1 + x_2
+ \lambda_1 \: ( 1 - x_0 - x_1 )^2
\\
&+ \lambda_2 \: \big[ 3 - ( 2 x_0 + 2 x_1 + x_2 )
- ( x_3 + 2 x_4 ) \big]^2
\\
&+ \lambda_3 \: \big[ ( x_0 + x_1 + x_2 )
- x_5 - 1 \big]^2,
\end{align*}
where the optimal solution is $(1, 0, 0)$.

In [ ]:
def custom_ham(lambda_vals, include_id=False):
    """
    Generates the Hamiltonian for the custom optimization problem.

    Arguments:
    lambda_vals -- A tuple/list of penalty parameters (lambda_1, lambda_2, lambda_3)
    include_id -- identity as symbol (True) or value 1 (False)

    Returns:
    H_total -- The Hamiltonian expression
    symbol_list -- List of variables used in the Hamiltonian
    """
    # Define symbols for variables x0 to x5
    symbol_list = list(sp.symbols('x0:6'))  # x0, x1, x2, x3, x4, x5
    x0, x1, x2, x3, x4, x5 = symbol_list

    # Unpack penalty parameters
    lambda_1, lambda_2, lambda_3 = lambda_vals

    # Identity (used if needed)
    Ident = sp.symbols(r'\mathbb{I}') if include_id else 1.

    # Objective function: Minimize x0 + x1 + x2
    H_prob = x0 + 2*x1 + x2

    # Penalty Term for Constraint 1: x0 + x1 = 1
    P1 = lambda_1 * (1 - x0 - x1)**2

    # Penalty Term for Constraint 2: 2x0 + 2x1 + x2 + x3 + 2x4 = 3
    P2 = lambda_2 * (3 - (2*x0 + 2*x1 + x2) - (x3 + 2*x4))**2

    # Penalty Term for Constraint 3: x0 + x1 + x2 - x5 = 1
    P3 = lambda_3 * ((x0 + x1 + x2) - x5 - 1)**2

    # Total Hamiltonian
    H_total = H_prob + P1 + P2 + P3

    # Since variables are binary (x_i^2 = x_i), substitute x_i^2 with x_i
    # Expand the Hamiltonian
    H_total = H_total.expand()

    # Create a list of x_i^2 terms
    sq_syms = [temp_sym**2 for temp_sym in symbol_list]

    # Create a substitution dictionary: x_i^2 -> x_i
    conv_dict = dict(zip(sq_syms, symbol_list))

    # Substitute x_i^2 with x_i
    H_total = H_total.subs(conv_dict)

    return H_total, symbol_list

In [ ]:
lambda_vals = (5, 5, 5)

bkp_fun1, bkp_list1 = custom_ham(lambda_vals)
bkp_fun1

60*x0*x1 + 30*x0*x2 + 20*x0*x3 + 40*x0*x4 - 10*x0*x5 - 49*x0 + 30*x1*x2 + 20*x1*x3 + 40*x1*x4 - 10*x1*x5 - 48*x1 + 10*x2*x3 + 20*x2*x4 - 10*x2*x5 - 29*x2 + 20*x3*x4 - 25*x3 - 40*x4 + 15*x5 + 55

Qubit Hamiltonian.

In [ ]:
bkp_list1 = binary_to_pauli_list(bkp_fun1, bkp_list1)
bkp_list1

[['ZZIIII', 15.0],
 ['ZIZIII', 7.5],
 ['ZIIZII', 5.0],
 ['ZIIIZI', 10.0],
 ['ZIIIIZ', -2.5],
 ['ZIIIII', -10.5],
 ['IZZIII', 7.5],
 ['IZIZII', 5.0],
 ['IZIIZI', 10.0],
 ['IZIIIZ', -2.5],
 ['IZIIII', -11.0],
 ['IIZZII', 2.5],
 ['IIZIZI', 5.0],
 ['IIZIIZ', -2.5],
 ['IIZIII', -5.5],
 ['IIIZZI', 5.0],
 ['IIIZII', -5.0],
 ['IIIIZI', -10.0],
 ['IIIIII', 32.0]]

In [ ]:
bkp_ham1 = qt.Qobj( qubit_op_to_ham(bkp_list1).full() )
evals, evecs = bkp_ham1.eigenstates()

In [ ]:
evals[:10]

array([1., 2., 2., 3., 6., 6., 6., 6., 7., 7.])

In [ ]:
for i in range(10):
    print(f"State {i}: ", find_basis_state(evecs[i]))

State 0:  100100
State 1:  010100
State 2:  101001
State 3:  011001
State 4:  001010
State 5:  100000
State 6:  100010
State 7:  100101
State 8:  010000
State 9:  010010


Let us map to Fock Hamiltonian.
The state that we are looking for is $| 1, 0, 4 \rangle$.

In [ ]:
bkp_part1 = [1, 2, 3,] # Qubit partition

bkp_ham2 = pauli_list_to_qudit_terms(bkp_list1, bkp_part1)
bkp_ham2

[['P0, P0, P0', 55.0],
 ['P0, P0, P1', 70.0],
 ['P0, P0, P2', 15.0],
 ['P0, P0, P3', 30.0],
 ['P0, P0, P4', 30.0],
 ['P0, P0, P5', 45.0],
 ['P0, P0, P6', 10.0],
 ['P0, P0, P7', 25.0],
 ['P0, P1, P0', 26.0],
 ['P0, P1, P1', 31.0],
 ['P0, P1, P2', 6.0],
 ['P0, P1, P3', 11.0],
 ['P0, P1, P4', 11.0],
 ['P0, P1, P5', 16.0],
 ['P0, P1, P6', 11.0],
 ['P0, P1, P7', 16.0],
 ['P0, P2, P0', 7.0],
 ['P0, P2, P1', 12.0],
 ['P0, P2, P2', 7.0],
 ['P0, P2, P3', 12.0],
 ['P0, P2, P4', 2.0],
 ['P0, P2, P5', 7.0],
 ['P0, P2, P6', 22.0],
 ['P0, P2, P7', 27.0],
 ['P0, P3, P0', 8.0],
 ['P0, P3, P1', 3.0],
 ['P0, P3, P2', 28.0],
 ['P0, P3, P3', 23.0],
 ['P0, P3, P4', 13.0],
 ['P0, P3, P5', 8.0],
 ['P0, P3, P6', 53.0],
 ['P0, P3, P7', 48.0],
 ['P1, P0, P0', 6.0],
 ['P1, P0, P1', 11.0],
 ['P1, P0, P2', 6.0],
 ['P1, P0, P3', 11.0],
 ['P1, P0, P4', 1.0],
 ['P1, P0, P5', 6.0],
 ['P1, P0, P6', 21.0],
 ['P1, P0, P7', 26.0],
 ['P1, P1, P0', 7.0],
 ['P1, P1, P1', 2.0],
 ['P1, P1, P2', 27.0],
 ['P1, P1, P3', 22.0],
 [

In [ ]:
len(bkp_ham2)

64

Optimization.

In [ ]:
nfocks = [4, 8]
ndepth = 10

en, Xvec, int_results = ecd_opt_vqe(bkp_ham1, ndepth, nfocks, maxiter=1000, method='BFGS',
                                    verb=1, threshold=1e-12)

<ipython-input-20-4065e9f46fb4>:66: RuntimeWarning: Method BFGS cannot handle bounds.
  result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds,


-------------------
iter: 10
fval: 14.917061422646563
-------------------
iter: 20
fval: 8.275668805530223
-------------------
iter: 30
fval: 5.042423537585548
-------------------
iter: 40
fval: 3.1252953448382708
-------------------
iter: 50
fval: 2.3967971698585533
-------------------
iter: 60
fval: 2.0104759011580957
-------------------
iter: 70
fval: 1.7828886372617596
-------------------
iter: 80
fval: 1.6399631762687468
-------------------
iter: 90
fval: 1.451091797084987
-------------------
iter: 100
fval: 1.3827776824586946
-------------------
iter: 120
fval: 1.318098238486178
-------------------
iter: 130
fval: 1.2489870195757415
-------------------
iter: 140
fval: 1.1969960781971476
-------------------
iter: 150
fval: 1.1540484959439696
-------------------
iter: 160
fval: 1.1346062008754172
-------------------
iter: 170
fval: 1.1136190824129857
-------------------
iter: 180
fval: 1.1007282170363426
-------------------
iter: 190
fval: 1.0953751726946124
-------------------
ite

In [ ]:
Xvec

array([ 2.71233640e+00, -1.61459476e-01,  6.19916181e-01,  1.60032748e-01,
       -7.50950840e-01,  2.40869702e+00,  3.16110615e+00,  2.47839573e+00,
        1.43479438e-02,  9.42754494e-01,  1.35330633e+00,  1.80555511e+00,
        1.94978052e-02,  1.31740753e+00,  2.73769934e+00,  2.34068543e+00,
        3.09164110e+00,  2.92153221e+00,  2.46041352e+00,  3.14496031e+00,
        9.51200172e-01,  6.78560963e-01,  2.50824741e+00,  4.02498440e+00,
       -1.94078516e-01,  1.34278954e+00,  2.23586678e+00,  1.99423185e+00,
        5.42343987e+00, -3.49797509e-02,  1.61698282e+00,  2.48960135e+00,
        1.25994488e+00,  5.13525093e-01,  1.89193458e+00,  7.78441545e-01,
        3.11115833e-01,  1.69548120e+00,  1.64830118e-01,  2.18924756e+00,
        3.67587312e+00,  5.90297709e-01, -1.89784466e+00,  1.20309219e+00,
        4.74812833e+00,  2.38389433e+00,  7.94875539e-04,  1.23368615e+00,
        1.65903961e+00,  2.98435418e+00,  1.95525973e+00,  2.54106568e+00,
        4.48722703e+00,  

In [ ]:
int_results[:20]

[(14.917061422646563,
  array([ 2.51341202,  0.60167935,  0.78854183,  0.85899093,  0.72779207,
          1.19521651,  2.24286442,  1.51665443,  0.72495026,  1.22949844,
          1.77410822,  0.97898341,  0.87454695,  1.50393752,  2.8627975 ,
          2.12089771,  2.69721697,  2.45147662,  2.97597573,  2.27227231,
          0.43792493,  1.00868746,  2.31894001,  0.82122363,  1.14638155,
          2.0359589 ,  1.86040348,  2.46355528,  3.46814855,  2.28504436,
          0.9440808 ,  3.08935883,  2.93668812,  0.409233  ,  1.71047918,
          0.77552308,  0.80840755,  1.74445727,  0.90958603,  1.85028669,
          1.95613842,  1.82564466,  0.48007315,  3.15427644,  2.98092811,
          1.77664424,  0.60872895,  0.47127565,  0.26822214,  0.89353114,
          2.50475248,  1.55669336,  3.28953241,  1.74042268,  2.0807749 ,
          2.66421112,  2.66338371,  2.8817639 ,  3.22863638,  1.05483593,
         -0.13107121,  1.9993827 ,  2.0784463 ,  1.47171458,  2.12727923,
          0.4603